# `mult` processor demo

In [ ]:
USE_PRIVATE_DISTRO = True

%tensorflow_version 2.x
if USE_PRIVATE_DISTRO:
    print("[INFO] Using private distro. Be careful.")
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -qU /content/drive/MyDrive/SMC\ 10/DDSP-10/dist/ddsp-1.2.0.tar.gz
else:
    !pip install -qU ddsp

# Ignore a bunch of deprecation warnings
import warnings
warnings.filterwarnings("ignore")

import ddsp
import ddsp.training
from ddsp.colab.colab_utils import play, specplot
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
SAMPLE_RATE = 48000
FRAME_RATE = 250
DURATION = 4

TIME_STEPS = FRAME_RATE * DURATION
N_SAMPLES = SAMPLE_RATE * DURATION

In [ ]:
carrier = ddsp.synths.Harmonic(n_samples=N_SAMPLES, 
                            sample_rate=SAMPLE_RATE, 
                            name='carrier')
lfo = ddsp.synths.Harmonic(n_samples=N_SAMPLES, 
                            sample_rate=SAMPLE_RATE, 
                            name='lfo')
mult = ddsp.processors.Mult(name='mult')

dag = [
  (carrier, ['carrier_gain', 'carrier_harmonic_amps', 'carrier_f0_hz']),
  (lfo, ['lfo_gain', 'lfo_harmonic_amps', 'lfo_f0_hz']),
  (mult, ['carrier/signal', 'lfo/signal']),
]

In [ ]:
n_harmonics = 1

# Create a sinusoid
carrier_gain = np.ones(TIME_STEPS)[np.newaxis, :, np.newaxis]
carrier_harmonic_amps = np.ones((TIME_STEPS, n_harmonics))[np.newaxis, :, :]
carrier_f0_hz = 440.0 * np.ones(TIME_STEPS)[np.newaxis, :, np.newaxis]

# Create an LFO
lfo_gain = np.ones(TIME_STEPS)[np.newaxis, : , np.newaxis]
lfo_harmonic_amps = np.ones((TIME_STEPS,n_harmonics))[np.newaxis, :, :]
lfo_f0_hz = 2.0 * np.ones(TIME_STEPS)[np.newaxis, :, np.newaxis]

inputs = {
    'carrier_gain': carrier_gain,
    'carrier_harmonic_amps': carrier_harmonic_amps,
    'carrier_f0_hz': carrier_f0_hz,
    'lfo_gain': lfo_gain,
    'lfo_harmonic_amps': lfo_harmonic_amps,
    'lfo_f0_hz': lfo_f0_hz,
}
inputs = {k: v.astype(np.float32) for k, v in inputs.items()}

processor_group = ddsp.processors.ProcessorGroup(dag=dag)
audio_out = processor_group(inputs)

play(audio_out, sample_rate=SAMPLE_RATE)
specplot(audio_out)